In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mar 17 2024

"""
import pandas as pd

'''
Read Data to get asset names
'''
file_path = "CBdata.xlsx"
xls = pd.read_excel(file_path, sheet_name=None, header=None,engine='openpyxl')
Name_df = xls["NaME"]

import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import pandas as pd
import scipy.stats as stats
import warnings
import math
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numba import jit
from scipy.optimize import differential_evolution

'''
define functions
'''

 
def create_list_with_step(x):
    start = 0.2*x
    end = 3 * x
    step = 0.05 * x
    list_with_step = [i for i in range(int(start * 10), int(end * 10) + 1, int(step * 10))]
    return [i / 10 for i in list_with_step]

def find_min_abs_second_element(lists):
    return min(lists, key=lambda x: abs(x[3]))

from functions import explicit_FD
from functions import find_between

尝试用DE来calibrate（没成功）

Grid Search

In [ ]:
import tqdm
for i in tqdm.tqdm(range(len(Name_df))):
    # get sheets
    sheetname=Name_df.iloc[i,0].replace("/", " ")
    d=Name_df.iloc[i,1]
    CB_df=xls[sheetname]
    # fixed parameter
    CR=CB_df.iloc[0,9]
    CP=CB_df.iloc[1,9]
    Coupon=CB_df.iloc[2,9]/100
    FCP=pd.to_datetime(CB_df.iloc[4,9])
    CFQ=CB_df.iloc[5,9]
    M=pd.to_datetime(CB_df.iloc[6,9])
    call_date = CB_df.iloc[0,12]
    call_price = CB_df.iloc[1,12]
    put_date = CB_df.iloc[2,12]
    put_price = CB_df.iloc[3,12]

    # clean and merge data
    Price_df= CB_df.iloc[9:, 0:2].dropna()
    IVOL_df=CB_df.iloc[9:, 5:7].dropna()
    CDS_df=CB_df.iloc[9:, 7:9].dropna()
    S_df=CB_df.iloc[9:, 9:11].dropna()
    
    Price_df= Price_df.rename(columns={0: "date", 1: 'bond_price'})
    CDS_df= CDS_df.rename(columns={7: "date", 8: 'CDS'})
    IVOL_df= IVOL_df.rename(columns={5: "date", 6: 'IVOL'})
    S_df= S_df.rename(columns={9: "date", 10: 'S'})
    Price_df['date'] = pd.to_datetime(Price_df['date'])
    IVOL_df['date'] = pd.to_datetime(IVOL_df['date'])
    CDS_df['date'] = pd.to_datetime(CDS_df['date'])
    S_df['date'] = pd.to_datetime(S_df['date'])
    Price_df = pd.merge(Price_df, IVOL_df, on="date", how='inner')
    Price_df = pd.merge(Price_df, CDS_df, on="date", how='inner')
    Price_df = pd.merge(Price_df, S_df, on="date", how='inner')
    
    r_df=xls['R'].rename(columns={0: "date"}).iloc[1:,:]
    r_df['date'] = pd.to_datetime(r_df['date']) 
    Price_df = pd.merge(Price_df, r_df, on="date", how='inner')
    Price_df =Price_df .dropna()

    if len(Price_df)>1:
        rlist=[1/12,1/6,1/4,1/2,1,2,3,5,7,10,20,30]
        SimulatePrice=[]
        for j in tqdm.tqdm(range(len(Price_df))):
            date1,v,CDS,S=Price_df.iloc[j,0],Price_df.iloc[j,2],Price_df.iloc[j,3],Price_df.iloc[j,4]

            # if there's no call provision or call provision passed, don't consider it
            if pd.isna(call_date):
                call_price = -1 
            else:
                if call_date<date1:
                    call_price = -1
                    
            if pd.isna(put_date):
                put_price = -1 
            else:
                if put_date<date1:
                    put_price = -1

            IVOLlist=create_list_with_step(v)
            T = (M - date1).days
            CPT=(FCP - date1).days%(360/CFQ)/T
            Year=T/360
            rlow,rhigh=find_between(rlist,Year)
            r=((Year-rlow)*Price_df.iloc[j,int(rlist.index(rlow))+5]/(rhigh-rlow)+(rhigh-Year)*Price_df.iloc[j,int(rlist.index(rhigh))+5]/(rhigh-rlow))/100
            cdsrange=[]

            for k in tqdm.tqdm(range(len(IVOLlist))):
                a=explicit_FD(100,Year,IVOLlist[k]/100,r,d,CDS/10000,CP,Coupon,CFQ,CPT,225,5000,call_price,put_price)
                S_app=round(math.log(S)/(math.log(450)/225))

                # calculate model price
                result=a[S_app,0]

                error = (result-Price_df.iloc[j,1])/Price_df.iloc[j,1]
                cdsrange.append([IVOLlist[k],r,error])

            a=find_min_abs_second_element(cdsrange)
            SimulatePrice.append(a)
            
        df = pd.DataFrame(SimulatePrice, columns=['BestIVOL','Interest Rate',"RelativeDifference"])
        a1 = pd.concat([Price_df, df], axis=1)
        a1.to_excel('TF calibration results with call and put/'+sheetname+'.xlsx', index=False)

  0%|          | 0/85 [00:00<?, ?it/s]




















100%|██████████| 58/58 [00:04<00:00, 13.21it/s]
















100%|██████████| 57/57 [00:01<00:00, 30.03it/s]
















100%|██████████| 58/58 [00:01<00:00, 30.37it/s]



















100%|██████████| 58/58 [00:02<00:00, 25.34it/s]




















100%|██████████| 57/57 [00:02<00:00, 23.36it/s]



















100%|██████████| 57/57 [00:02<00:00, 24.89it/s]


















100%|██████████| 57/57 [00:02<00:00, 27.73it/s]


















100%|██████████| 57/57 [00:02<00:00, 27.60it/s]




















100%|██████████| 57/57 [00:02<00:00, 23.48it/s]






















100%|██████████| 58/58 [00:02<00:00, 20.71it/s]

















100%|██████████| 57/57 [00:01<00:00, 29.87it/s]

















100%|██████████| 57/57 [00:01<00:00, 29.19it/s]
















100%|██████████| 58/58 [00:01<00:00, 29.46it/s]



















100%|██████████| 57/57 [00:02<00:00, 26.04it/s]



















100%|██████████| 57/57 [00:02<00:00, 27.39it/s]





